## Load US CENSUS All-Sector Report data from 2017 datasets (see https://data.census.gov/cedsci/ and set your own parameters) for original sources

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : https://redhat-osc-physical-landing-647521352890.s3.amazonaws.com/US_CENSUS/US_Census-All-Sector-Report-2017/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

## Based on OSC Ingest Tutorial by Erik Erlandson (GitHub: erikerlandson)

## python dependencies
The following are python packages commonly used for data ingest.
If your jupyter environment does not already have these,
you can copy and paste these into a live code cell to install them.

Run these in a notebook cell if you need to install onto your nb env
```
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

## Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [ ]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

## S3 and boto3

In [ ]:
import boto3
s3_src = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
src_bucket = s3_src.Bucket(os.environ['S3_LANDING_BUCKET'])

s3_trino = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)
trino_bucket = s3_trino.Bucket(os.environ["S3_DEV_BUCKET"])

## Connecting to Trino with sqlalchemy

In [ ]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev',
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

## tutorial example data

In [ ]:
import pandas as pd

data_dictionary = 'ECNCOMP2017.EC1700COMP_metadata_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{data_dictionary}')
bObj.download_file(f'/tmp/ingest-{data_dictionary}')
dd_df = pd.read_csv(f'/tmp/ingest-{data_dictionary}', sep=',', header=None, engine='c')
dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{data_dictionary}')
dd_df

In [ ]:
sic_codes = 'sic88_97.txt'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{sic_codes}')
bObj.download_file(f'/tmp/ingest-{sic_codes}')
df = pd.read_csv(f'/tmp/ingest-{sic_codes}', sep='  ', header=None, names=['sic', 'desc'], engine='python')
os.unlink(f'/tmp/ingest-{sic_codes}')

df.drop(df.index[0], inplace=True)
sic2_df = df.loc[df.sic.str.contains(r'^\d\d--$')]
df = df.loc[df.sic.str.contains(r'^\d{4}$')]
df.sic = df.sic.astype('int32')
sic2_df.sic = sic2_df.sic.str[0:2].astype('int32')
df.reset_index (drop=True, inplace=True)
sic2_df.reset_index (drop=True, inplace=True)

sic4_df = df

sic4_df.info(verbose=True)
sic2_df.info(verbose=True)

In [ ]:
survey_report = 'ECNCOMP2017.EC1700COMP_data_with_overlays_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{survey_report}')
bObj.download_file(f'/tmp/ingest-{survey_report}')
df = pd.read_csv(f'/tmp/ingest-{survey_report}', sep=',', header=0, engine='c')
os.unlink(f'/tmp/ingest-{survey_report}')

df = df.drop(df.index[0])
df.reset_index (drop=True, inplace=True)
df

## `osc-ingest-tools`

## Enforcing valid SQL column names

In [ ]:
import osc_ingest_trino as osc
from io import BytesIO

osc.enforce_sql_column_names(df, inplace=True)
df

## Trino catalog, schema, table

In [ ]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'us_census'
ingest_table = 'all_sector_survey_2017'
dd_table = 'data_dictionary'
sic4_table = 'sic4'
sic2_table = 'sic2'

## Staging parquet files for trino tables

In [ ]:
buf = BytesIO()
df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{ingest_table}/data.parquet')

## Declaring a trino table on top of a raw parquet file

In [ ]:
qres = engine.execute(f'create schema if not exists {ingest_schema}')
print(qres.fetchall())

columnschema = osc.create_table_schema_pairs(df)

tabledef = f"""
create table if not exists {ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{ingest_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

In [ ]:
buf = BytesIO()
dd_df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{dd_table}/dd.parquet')

In [ ]:
columnschema = osc.create_table_schema_pairs(dd_df)

tabledef = f"""
create table if not exists {ingest_schema}.{dd_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{dd_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

In [ ]:
buf = BytesIO()
sic4_df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{sic4_table}/dd.parquet')

In [ ]:
columnschema = osc.create_table_schema_pairs(sic4_df)

tabledef = f"""
create table if not exists {ingest_schema}.{sic4_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{sic4_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

In [ ]:
buf = BytesIO()
sic2_df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{sic2_table}/dd.parquet')

In [ ]:
columnschema = osc.create_table_schema_pairs(sic2_df)

tabledef = f"""
create table if not exists {ingest_schema}.{sic2_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{sic2_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

## SQL queries on our new table

In [ ]:
sql = f"""
select * from {ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine).convert_dtypes()